### 다중분류(Multi-Class Classification)
---
- 3 개 이상의 답 중 1개를 고르는 경우
- 비용함수(Cost Function)  => Cross Entropy
    * 이진 교차 엔트로피(Binary Cross Entropy)와 유사한 방식으로 계산

In [1]:
# 모듈 로딩 ------------------------------
import torch
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

In [3]:
# 데이터 생성 -----------------------------
z = torch.FloatTensor([1, 2, 3])

print(z.shape, f'{z.ndim}차원', z)

torch.Size([3]) 1차원 tensor([1., 2., 3.])


In [4]:
# 소프트맥스 함수 적용 ---------------------
# softmax(입력 tensor dim=softmax수행할 차원)
hypothesis = F.softmax(z, dim=0)

print(hypothesis, hypothesis.sum())

tensor([0.0900, 0.2447, 0.6652]) tensor(1.)


In [5]:
z2 = torch.rand(3, 5, requires_grad=True)
print(z2.shape, f'{z2.ndim}차원', z2, sep='\n')

torch.Size([3, 5])
2차원
tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)


In [6]:
# dim=1 ==> (3, 5)에서 5가지에 대한 값 
hypothesis = F.softmax(z2, dim=1)

print(hypothesis, hypothesis.sum())

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>) tensor(3.0000, grad_fn=<SumBackward0>)


### One-Hot-Encoding

- squeeze() / unsqueeze() 살펴보기

In [7]:
y2=y.unsqueeze(1)
y2, y2.shape

NameError: name 'y' is not defined

In [ ]:
# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis) 
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.]])

In [ ]:
# 1차원 모두 제거 ------------------------------------
#  torch.rand() : 0과 1 사이의 숫자를 균등하게 생성
x = torch.rand(1, 1, 20, 128)
x.shape, x.ndim

(torch.Size([1, 1, 20, 128]), 4)

In [ ]:
x1 = x.squeeze() #[1, 1, 20, 128] -> [20, 128]
x1.shape, x1.ndim

(torch.Size([20, 128]), 2)

In [ ]:
x2 = x.squeeze(dim=0) #[1, 1, 20, 128] -> [1, 20, 128]
x2.shape, x2.ndim

(torch.Size([1, 20, 128]), 3)

In [ ]:
xx = torch.rand(3, 20, 128)
xx.shape, xx.ndim

(torch.Size([3, 20, 128]), 3)

In [ ]:
xx1 = xx.unsqueeze(dim=1) #[3, 20, 128] -> [3, 1, 20, 128]
xx1.shape, xx1.ndim

(torch.Size([3, 1, 20, 128]), 4)

In [ ]:
xx2 = xx.unsqueeze(dim=0) #[3, 20, 128] -> [3, 1, 20, 128]
xx2.shape, xx2.ndim

(torch.Size([1, 3, 20, 128]), 4)

- One Hot Encoding

In [ ]:
# 임의의 라벨 데이터 생성
y = torch.randint(5, (3,)).long()

y, y.shape

(tensor([0, 1, 4]), torch.Size([3]))

In [ ]:
y.unsqueeze(1)

tensor([[0],
        [1],
        [4]])

In [ ]:
# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis) 

y_one_hot

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [ ]:
#dim=axis, index, src, reduce=None
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])

## nn.Module 활용 구현
---

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
torch.manual_seed(1)

In [ ]:
# 데이터 준비 ------------------------------
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [ ]:
# 데이터 준비 ------------------------------
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [1, 1, 1, 0, 0, 0, 1, 1]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [ ]:
print(x_train.shape,  y_train.shape)

torch.Size([8, 4]) torch.Size([8])


In [ ]:
# One-Hot-Encoding 라벨
#y_one_hot = torch.zeros(8, 3)
y_one_hot = torch.zeros(8, 2)
print(y_one_hot)
print(y_train.unsqueeze(1))


tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1]])


In [ ]:
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape, y_one_hot, sep='\n')

torch.Size([8, 2])
tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.]])


In [ ]:
# 모델 설계 및 초기화 ----------------------------
# 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3.
model = nn.Linear(4, 3)

# 4개의 특성을 가지고 2개의 클래스로 분류. input_dim=4, output_dim=2.
model = nn.Linear(4, 2)

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
# 학습 -----------------------------------------------
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/1000 Cost: 2.572310
Epoch  100/1000 Cost: 0.820514
Epoch  200/1000 Cost: 0.810985
Epoch  300/1000 Cost: 0.808653
Epoch  400/1000 Cost: 0.807723
Epoch  500/1000 Cost: 0.807374
Epoch  600/1000 Cost: 0.807326
Epoch  700/1000 Cost: 0.807440
Epoch  800/1000 Cost: 0.807637
Epoch  900/1000 Cost: 0.807874
Epoch 1000/1000 Cost: 0.808126
